In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.56 requires cryptography<40.0.2,>=40.0.0, but you have cryptography 44.0.2 which is incompatible.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.


In [2]:
from odp.client import OdpClient # The SDK
import pandas as pd

In [3]:
client = OdpClient()

In [4]:
dataset = client.catalog.get(("8131a076-af24-483b-b9a1-20a37433ef46")) ## UUID of ProtectedSeas dataset from the catalog
dataset.metadata.display_name

'ProtectedSeas MPA Dataset'

In [5]:
# PS_data = client.tabular.select_as_list(dataset, filter_query={'#ST_WITHIN': ['$geometry', query_geometry.value]}, limit=10000)

In [6]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import transform
import pyproj

# Define the point
center = Point(-64.385888, 48.987655)  # Longitude first, then latitude

# Define the projection for buffer calculations (to convert degrees to meters)
wgs84 = pyproj.CRS("EPSG:4326")  # Standard lat/lon
utm = pyproj.CRS("EPSG:32652")  # Approximate UTM zone for this region
project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
reverse_project = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform

# Convert point to UTM, buffer in meters, then convert back
utm_point = transform(project, center)
buffered = utm_point.buffer(50000)  # 50km buffer
circle_polygon = transform(reverse_project, buffered)

# Convert to GeoJSON-like format
query_geometry = {
    "type": "Polygon",
    "coordinates": [list(circle_polygon.exterior.coords)]
}

# Run the query
PS_data = client.tabular.select_as_list(
    dataset, 
    filter_query={'#ST_WITHIN': ['$geometry', query_geometry]},
    limit=10000
)


In [10]:
# PS_data

In [7]:
PS_df = pd.DataFrame(PS_data)  # Convert list to DataFrame
PS_df.head(5)

,regulation_url,boundary_source,inshore_only,definitions,anchoring_prohibited,construction_prohibited,hook_n_line,dredging_prohibited,trolling,tribal,...,site_name,coastline_match,latest_updates,industrial_or_mineral_exploration_prohibited,season,geometry_reduced,state,longlining,entry_prohibited,regulation_name
0,https://laws-lois.justice.gc.ca/eng/regulation...,Parks Canada,3.0,None,3,3,2,3,1,1,...,Forillon National Park,1.0,"Check Fisheries and Oceans Canada's <a href=""h...",3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",QC,1,3,National Parks of Canada Fishing Regulations (...


In [8]:
columns_to_keep = [
    "site_name",
    "purpose",
    "species_of_concern",
    "restrictions",
    "iucn_cat",
    "regulation_name",
    "managing_authority",
    "country",
    "geometry_reduced"
]

# Filter and reorder the DataFrame
PS_df = PS_df[columns_to_keep]

# Display the first 5 rows
PS_df.head(5)

,site_name,purpose,species_of_concern,restrictions,iucn_cat,regulation_name,managing_authority,country,geometry_reduced
0,Forillon National Park,To protect ecosystems providing habitat for di...,"Minke whale, Atlantic white-sided dolphin, hum...",1. Catch limits and closures as set out in Nat...,Unassigned,National Parks of Canada Fishing Regulations (...,Parks Canada,Canada,"{'type': 'MultiPolygon', 'coordinates': [[[[-6..."
